<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Wordnet" data-toc-modified-id="Wordnet-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Wordnet</a></span></li><li><span><a href="#PPMI" data-toc-modified-id="PPMI-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>PPMI</a></span></li><li><span><a href="#Autres-mesures-possibles" data-toc-modified-id="Autres-mesures-possibles-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Autres mesures possibles</a></span><ul class="toc-item"><li><span><a href="#Word-Embeddings" data-toc-modified-id="Word-Embeddings-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Word Embeddings</a></span></li><li><span><a href="#Matrice-mots/documents" data-toc-modified-id="Matrice-mots/documents-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Matrice mots/documents</a></span></li><li><span><a href="#Formule-ESA" data-toc-modified-id="Formule-ESA-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Formule ESA</a></span></li></ul></li></ul></div>

# Présentation rapide des similarités

## Wordnet

Wordnet est une ressource sémantique externe à nos données construites manuellement qui contient un grand nombre de mots (noms, verbes, adjectifs...) et qui constitue donc une base très riche. Les mots sont organisés en synsets (sortes d'arbres) et pour retrouver une similarité entre 2 mots, wordnet va chercher le chemin le plus court pour aller d'un synset à un autre. Le but des ressources sémantiques est d'associer des mots d'un texte à leur sens, d'identifier des similarités sémantiques entre mots, concepts et textes ainsi que d'associer des entités présentes dans le texte à des objets du monde réel. Voici un exemple d'implémentation de la similarité en utilisant la ressource wordnet :

In [1]:
import numpy as np
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alice\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Le crochet 0 montre le premier sens du mot voiture
word1 = wordnet.synsets("car")[0]
word2 = wordnet.synsets("cat")[0]

print("Similarité wordnet entre les mots chat et voiture: ",
      word1.path_similarity(word2))

Similarité wordnet entre les mots chat et voiture:  0.05555555555555555


In [3]:
mot1 = wordnet.synsets("man")[0]
mot2 = wordnet.synsets("women")[0]

print("Similarité wordnet entre les mots homme et femme: ",
      mot1.path_similarity(mot2))

Similarité wordnet entre les mots homme et femme:  0.3333333333333333


On peut voir que malgré que les mots car et cat soient très proches syntaxiquement, ils sont bien moins proches sémantiquement (la sémantique prend en compte le sens du texte et non pas la syntaxe) que les mots hommes et femmes.
En savoir plus sur wordnet : https://wordnet.princeton.edu/

Vous pouvez tester d'autres similarités entre mots et voir quelles autres ressources sémantiques existent.

## PPMI

Le PPMI est une approche basée sur la sémantique distributionnelle. Le but du PPMI est d'essayer de voir quels mots apparaissent plutôt dans quel contexte et si les mots apparaissent souvent dans le même contexte alors on dira qu'ils sont similaires au niveau sémantique. Pour cela, on doit calculer la distribution d'apparitions des mots dans le corpus de texte puis créer un vecteur de sa distribution dans le corpus. Un exemple simple va être présenté ci-dessous, avec une seule phrase qui servira je l'espère à donner une idée générale de l'utilisation de cette méthode. La phrase va être divisée en plusieurs fenêtres de taille égale qui représenteront chacune un contexte de mots. **Attention**, le code est surement compliqué à lire et je préfère ne pas détailler les formules derrière pour éviter de perdre tout le monde donc si cela vous parait trop compliqué, n'hésitez pas à vous concentrer uniquement sur les idées voire à passer cette étape.

In [4]:
def fenetre_glisant(text: str, size: int)->list:
    """Documentation
    Parametres:
        text (string): text to analyze
        size (integer): window size

    Out:
        results (list): list of 5 words
    """

    text = text.lower()  # on met tout le texte en minuscule
    text = text.replace(',', ' ').replace(
        '.', ' ').replace('(', ' ').replace(')', ' ')
    # remplacement de caractères spéciaux

    partes = text.split()  # on découpe le texte en mots
    results = []
    n = len(partes) - (size - 1)
    for index in range(n):
        results.append(partes[index:index + size])
    return results


montexte = "Le PPMI permet des similarités vectorielle entre les mots et peut être utile pour identifier les similarités entre les mots du corpus"
print(fenetre_glisant(montexte, 5))

[['le', 'ppmi', 'permet', 'des', 'similarités'], ['ppmi', 'permet', 'des', 'similarités', 'vectorielle'], ['permet', 'des', 'similarités', 'vectorielle', 'entre'], ['des', 'similarités', 'vectorielle', 'entre', 'les'], ['similarités', 'vectorielle', 'entre', 'les', 'mots'], ['vectorielle', 'entre', 'les', 'mots', 'et'], ['entre', 'les', 'mots', 'et', 'peut'], ['les', 'mots', 'et', 'peut', 'être'], ['mots', 'et', 'peut', 'être', 'utile'], ['et', 'peut', 'être', 'utile', 'pour'], ['peut', 'être', 'utile', 'pour', 'identifier'], ['être', 'utile', 'pour', 'identifier', 'les'], ['utile', 'pour', 'identifier', 'les', 'similarités'], ['pour', 'identifier', 'les', 'similarités', 'entre'], ['identifier', 'les', 'similarités', 'entre', 'les'], ['les', 'similarités', 'entre', 'les', 'mots'], ['similarités', 'entre', 'les', 'mots', 'du'], ['entre', 'les', 'mots', 'du', 'corpus']]


In [5]:
z = fenetre_glisant(montexte, 5)
# fonction pour calculer la frequence entre 2 mots avec une fenetre glissante de taille fixe


def p(word1: str, word2: str, window: list)->float:
    """Documentation
    Parametres:
        word1 (string): a word
        word2 (string): a word
        window (list): calls the function : "fenetreGlisant"

    Out:
        results (float): a similarity score
    """

    account = 0
    n = len(window)
    for i in range(n):
        if ((word1 in window[i]) and (word2 in window[i])):
            account = account + 1
    if (account != 0):
        return (account/(n))
    else:
        return 0


p("similarités", "mots", z)

0.16666666666666666

In [6]:
text = montexte.lower()
text = text.replace(',', ' ').replace(
    '.', ' ').replace('(', ' ').replace(')', ' ')

words = text.split()

In [7]:
def calculerPPMI(px: float, py: float, word1: str, word2: str, window: list)->float:  # PPMI
    """Documentation
    Parametres:
        px (float): calls the function : "calculerpx"
        py (float): calls the function : "calculerpx"
        word1 (string): a word
        word2 (string): a word
        window (list): calls the function : "fenetreGlisant"

    Out:
        result (float): a similarity score between 2 words
    """

    t = p(word1, word2, window)
    if ((px*py > 0) and (np.log(t/(px*py)) > 0)):
        return np.log(t/(px*py))
    else:
        return 0


def calculerpx(word: str, windows: list)->float:  # px
    """Documentation
    Parametres:
        word (string): a word
        window (list): calls the function : "fenetreGlisant"

    Out:
        result (float): word appearance score
    """

    compt = 0
    for i in range(len(windows)):
        if word in windows[i]:
            compt = compt + 1
    n = compt/len(windows)
    return n


# Application de la fonction pour calculer les valeurs du PPMI sur nos fenetres glissantes
windows = fenetre_glisant(montexte, 5)
for word1 in words:
    for word2 in words:
        print(word1, word2, calculerPPMI(calculerpx(word1, windows),
                                         calculerpx(word2, windows), word1, word2, windows))

le le 2.8903717578961645
le ppmi 2.1972245773362196
le permet 1.791759469228055
le des 1.5040773967762742
le similarités 0.5877866649021191
le vectorielle 0
le entre 0
le les 0
le mots 0
le et 0
le peut 0
le être 0
le utile 0
le pour 0
le identifier 0
le les 0
le similarités 0.5877866649021191
le entre 0
le les 0
le mots 0
le du 0
le corpus 0
ppmi le 2.1972245773362196
ppmi ppmi 2.1972245773362196
ppmi permet 1.791759469228055
ppmi des 1.5040773967762742
ppmi similarités 0.5877866649021191
ppmi vectorielle 0.5877866649021191
ppmi entre 0
ppmi les 0
ppmi mots 0
ppmi et 0
ppmi peut 0
ppmi être 0
ppmi utile 0
ppmi pour 0
ppmi identifier 0
ppmi les 0
ppmi similarités 0.5877866649021191
ppmi entre 0
ppmi les 0
ppmi mots 0
ppmi du 0
ppmi corpus 0
permet le 1.791759469228055
permet ppmi 1.791759469228055
permet permet 1.791759469228055
permet des 1.5040773967762742
permet similarités 0.5877866649021191
permet vectorielle 0.8754687373538999
permet entre 0
permet les 0
permet mots 0
permet et 0

C:\Users\alice\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


 0
corpus identifier 0
corpus les 0.4054651081081644
corpus similarités 0
corpus entre 0.5877866649021191
corpus les 0.4054651081081644
corpus mots 0.8109302162163288
corpus du 2.1972245773362196
corpus corpus 2.8903717578961645


Un exemple intéressant est donné sur ce lien https://web.stanford.edu/~jurafsky/li15/lec3.vector.pdf entre les diapositives 20 et 31 de la façon dont sont calculer ces valeur.

Vous pouvez vous amuser à changer les paramètres du PPMI ou à tester d'autres exemples si vous souhaitez mieux comprendre. Pour ceux qui comprennent aisément, n'hésitez pas à changer les formules de calcul de calculerpx et calculerPPMI pour d'une part prendre confiance avec ce système de calcul et d'une seconde part voir d'autres manières de calculer ces similarités. 

## Autres mesures possibles

### Word Embeddings

Cette méthode est basée sur les réseaux de neurones et permet d'avoir une représentation des vecteurs de mots. Normalement un lien parlant de cette méthode a du vous être envoyé.

Plusieurs méthodes telles que les similarités cosinus (https://fr.wikipedia.org/wiki/Similarit%C3%A9_cosinus) peuvent être utilisées afin de calculer les similarités entre les vecteurs de mots.

### Matrice mots/documents

Des vecteurs peuvent être calculés en utilisant une simple matrice contenant en colonne les différents documents que l'on possède et en ligne les mots. Chaque mot aura une valeur de fréquence d'apparition dans chaque document et donc chaque ligne donnera un vecteur de mots avec un nombre de valeur égal au nombre de documents que l'on possède. On peut ensuite calculer des similarités vectorielles avec la formule de notre choix entre chaque mot. Cette méthode est une méthode dite basée contenue car elle s'appuie uniquement sur notre corpus pour construire ces similarités entre mots.

### Formule ESA

Cette formule est une formule qui permet une représentation des textes basés contenus une nouvelle fois. Elle est utilisée pour divers sujets de textes et NLP et peut-être utile pour notre sujet. Voici un lien Wikipedia expliquant rapidement son fonctionnement : https://en.wikipedia.org/wiki/Explicit_semantic_analysis

# Conclusion

Le principe de similarité va principalement être utilisé par le sous-groupe traitant les doublons sémantiques mais risque d'être utile à l'ensemble des collaborateurs du groupe de classification. Les similarités sémantiques sont de nos jours utilisés dans les systèmes de reconnaissance de plagiat (notamment ceux utilisé par la Fac qui est d'ailleurs très performante) et peuvent s'avérer utile dans de nombreux sujets concernant la NLP (Natural Language Processing). Les façons de calculer des similarités sont très variées et on espère pouvoir utiliser un maximum d'entre eux afin de répondre aux problèmes des doublons sémantiques.

Envoyez un mail à jeremy.johann@univ-tlse3.fr ou lilian.dulinge@univ-tlse3.fr ou alice.airault@univ-tlse3.fr pour nous dire si vous avez plutôt bien compris ce sujet ou pas du tout et si vous souhaitez poser des questions.
Bonne lecture et bonne chance.